In [ ]:
!pip install torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
import sys
%cd drive/MyDrive/
%ls
HOME = os.getcwd()
print(os.listdir(HOME))
sys.path.append(HOME)

# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
import torchtext
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation
from utils import train, evaluate, set_seed_nb, unit_test_values, deterministic_init, plot_curves

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/'
/content/drive/MyDrive
 checkpoint_multihead_attention_ce.pt   HW1@
 checkpoint_nbow_hinge.pt               model_epoch_0.pt
 checkpoint_nbow_perceptron.pt         'model_training (1).cpython-310.pyc'
'Colab Notebooks'/                     'model_training (2).cpython-310.pyc'
 CS7643Project@                        'model_training (3).cpython-310.pyc'
'CS 8803-CAI'/                          model_training.cpython-310.pyc
 data/
['CS7643Project', 'data', 'Colab Notebooks', 'HW1', 'checkpoint_multihead_attention_ce.pt', 'checkpoint_nbow_perceptron.pt', 'checkpoint_nbow_hinge.pt', 'model_training (3).cpython-310.pyc', 'model_training (2).cpython-310.pyc', 'model_training (1).cpython-310.pyc', 'model_training.cpython-310.pyc', 'model_epoch_0.pt', 'CS 8803-CAI']


AttributeError: partially initialized module 'torch' has no attribute 'nn' (most likely due to a circular import)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
folder_path = '/content/drive/MyDrive/CS 8803-CAI/output_npy'

In [ ]:
import numpy as np
import torch
import librosa
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset

# Define data path and emotion mapping
data_path = folder_path
label_mapping = {'happy': 0, 'exciting': 1, 'scary': 2, 'funny': 3, 'sad': 4, 'tender': 5, 'angry': 6}

# Lists to store data and labels
spectrograms = []
labels = []

# Process each file
for file in os.listdir(data_path):
    if file.endswith(".npy"):
        # Load data and convert to spectrogram
        data = np.load(os.path.join(data_path, file))
        spectrogram = librosa.feature.melspectrogram(y=data, sr=22050, n_mels=128)

        # Normalize and add a channel dimension for compatibility with ResNet
        spectrogram = torch.tensor(spectrogram, dtype=torch.float32).unsqueeze(0)
        spectrograms.append(spectrogram)

        # Extract label from filename
        emotion = file.split('_')[1]
        labels.append(label_mapping[emotion])

# Convert to tensors
X = torch.stack(spectrograms)
y = torch.tensor(labels)

# Create DataLoader
dataset = TensorDataset(X, y)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Modify input layer for 1-channel input
model.fc = nn.Linear(model.fc.in_features, 7)  # Modify output layer to match the 7 emotion classes

In [ ]:
# Set device and move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader):.4f}")

In [ ]:
torch.save(model.state_dict(), "emotion_resnet_model.pth")